In [38]:
import os
import pymongo
import pandas as pd
import json
from datetime import datetime as dt
from src.features.dedup_model import load_results
from src.data.utils import load_json_as_df, create_dir
from src.data.image_utils import download_images_from_group
import src.models.nima_model as nima 



In [2]:
from src.constants import (
    RAW_DIR_PATH,
    INTERIM_DIR_PATH,
    PROCESSED_DIR_PATH
)

In [3]:
DB_USERNAME = os.getenv("MONGO_USERNAME")
DB_PASSWORD = os.getenv("MONGO_PASSWORD")
DB_URL = os.getenv("MONGO_URL")
DB_NAME = os.getenv("MONGO_DB")
DB_NAME

'mlops-project'

In [4]:
client = pymongo.MongoClient(
    f"mongodb+srv://{DB_USERNAME}:{DB_PASSWORD}@{DB_URL}/?retryWrites=true&w=majority"
)
client

MongoClient(host=['ac-52gsxzj-shard-00-01.yi2z9ch.mongodb.net:27017', 'ac-52gsxzj-shard-00-02.yi2z9ch.mongodb.net:27017', 'ac-52gsxzj-shard-00-00.yi2z9ch.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-12itgl-shard-0', tls=True)

In [5]:
db = client[DB_NAME]
db.head

Collection(Database(MongoClient(host=['ac-52gsxzj-shard-00-01.yi2z9ch.mongodb.net:27017', 'ac-52gsxzj-shard-00-02.yi2z9ch.mongodb.net:27017', 'ac-52gsxzj-shard-00-00.yi2z9ch.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-12itgl-shard-0', tls=True), 'mlops-project'), 'head')

In [6]:
db.list_collection_names()


['duplicated_images_model_results']

In [7]:
collectionDuplicated = db['duplicated_images_model_results']
df = pd.DataFrame(collectionDuplicated.find())

In [8]:
df.head()

,_id,date,group_id,base_image_id,duplicated_image_id,probability
0,63c530068113af969cb993ec,2023-01-16 11:07:50.773,102292696432481,1631,1703,0.916148
1,63c530068113af969cb993ed,2023-01-16 11:07:50.774,102292696432481,1631,1710,0.868106
2,63c530068113af969cb993ee,2023-01-16 11:07:50.774,102292696432481,1631,1741,0.868106
3,63c530068113af969cb993ef,2023-01-16 11:07:50.774,102292696432481,1631,1744,0.868106
4,63c530068113af969cb993f0,2023-01-16 11:07:50.774,102292696432481,1631,1760,1.000000


In [18]:
len(df.index)

26126

In [9]:
images_json_file_name = "images.json"
images_json_file_path = os.path.join(RAW_DIR_PATH, images_json_file_name)

In [10]:
images_df = load_json_as_df(images_json_file_path, index_col="image_id")
images_df.head()

,group_id,image_url
image_id,,
1,2224086878288,https://i.travelapi.com/hotels/57000000/569800...
2,2224086878288,https://i.travelapi.com/hotels/57000000/569800...
3,2224086878288,https://i.travelapi.com/hotels/57000000/569800...
4,2224086878288,https://i.travelapi.com/hotels/57000000/569800...
5,2224086878288,https://i.travelapi.com/hotels/57000000/569800...


In [19]:
len(images_df.index)

65795

In [31]:
df1 = images_df[images_df['group_id']=='194872942013291']
df1.head()

,group_id,image_url
image_id,,
3128,194872942013291,https://q-xx.bstatic.com/xdata/w/hotel/max500_...
3129,194872942013291,https://q-xx.bstatic.com/xdata/w/hotel/max500_...
3130,194872942013291,https://q-xx.bstatic.com/xdata/w/hotel/max500_...
3131,194872942013291,https://q-xx.bstatic.com/xdata/w/hotel/max500_...
3132,194872942013291,https://q-xx.bstatic.com/xdata/w/hotel/max500_...


In [12]:
images_dir_path = os.path.join(INTERIM_DIR_PATH, "images")
create_dir(images_dir_path)

In [23]:
for group_id, df_group in df.groupby('group_id'):

    lista = df_group['base_image_id'].to_list() +df_group['duplicated_image_id'].to_list()
    mylist = list(dict.fromkeys(lista))

    df_images_group= pd.DataFrame (mylist, columns = ['image_id'])
    df_images_group['group_id'] = group_id

    merged_group = pd.merge(left=df_images_group, right=images_df, left_on=['group_id','image_id'], right_on=['group_id','image_id'])

    download_images_from_group(images_df, group_id, (images_dir_path ))
    



365it [00:00, 5663.34it/s]
215it [00:00, 4188.36it/s]
25it [00:00, 822.92it/s]
125it [00:00, 3162.27it/s]
132it [00:00, 3904.18it/s]
40it [00:00, 2230.24it/s]
5it [00:00, 499.56it/s]
230it [00:00, 5108.07it/s]
210it [00:00, 2684.31it/s]
330it [00:00, 5506.99it/s]
225it [00:00, 5278.45it/s]
170it [00:00, 3937.03it/s]
345it [00:00, 6691.12it/s]
120it [00:00, 2877.36it/s]


In [13]:
for group_id, df_group in df.groupby('group_id'):

    lista = df_group['base_image_id'].to_list() +df_group['duplicated_image_id'].to_list()
    mylist = list(dict.fromkeys(lista))

    print(len(mylist))

365
215
25
125
132
40
5
230
210
330
225
170
345
120


In [14]:
model= "MobileNet"
src = os.path.join('/app','src')
features = os.path.join(src,'features')
nima_model_folder = os.path.join(features,'nima_model')
weights = os.path.join(nima_model_folder,'weights_mobilenet_aesthetic_0.07.hdf5')



In [25]:
images_dir_path

'/app/data/interim/images'

In [30]:
lista = []
for group_id, df_group in df.groupby('group_id'):
    image_folder = os.path.join(images_dir_path,group_id)
    predicted =  nima.predictImages(model,weights,image_folder)
    for row in predicted:
        row['group_id'] = group_id
    lista = lista + predicted    

/app/src/models/nima_model/__init__.py:27: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  return model.predict_generator(data_generator, workers=8, use_multiprocessing=True, verbose=1)
/usr/local/lib/python3.10/site-packages/keras/engine/data_adapter.py:925: RuntimeWarning: overflow encountered in cast
  return np.array(t, dtype=backend.floatx())


6/6 [==============================] - 10s 1s/step
<class 'list'>
4/4 [==============================] - 7s 1s/step
<class 'list'>
1/1 [==============================] - 2s 2s/step
<class 'list'>
2/2 [==============================] - 4s 1s/step
<class 'list'>
3/3 [==============================] - 4s 809ms/step
<class 'list'>
1/1 [==============================] - 2s 2s/step
<class 'list'>
1/1 [==============================] - 1s 1s/step
<class 'list'>
4/4 [==============================] - 7s 1s/step
<class 'list'>
4/4 [==============================] - 6s 1s/step
<class 'list'>


/usr/local/lib/python3.10/site-packages/keras/engine/data_adapter.py:925: RuntimeWarning: invalid value encountered in cast
  return np.array(t, dtype=backend.floatx())


6/6 [==============================] - 9s 1s/step
<class 'list'>
4/4 [==============================] - 6s 1s/step
<class 'list'>
3/3 [==============================] - 6s 1s/step
<class 'list'>
6/6 [==============================] - 10s 1s/step
<class 'list'>
2/2 [==============================] - 4s 1s/step
<class 'list'>


In [36]:
lista[0]

{'image_id': '1631',
 'mean_score_prediction': 5.60638951636065,
 'group_id': '102292696432481'}

In [39]:
print(json.dumps(lista, indent=2))

[
  {
    "image_id": "1631",
    "mean_score_prediction": 5.60638951636065,
    "group_id": "102292696432481"
  },
  {
    "image_id": "1632",
    "mean_score_prediction": 5.142864143301267,
    "group_id": "102292696432481"
  },
  {
    "image_id": "1633",
    "mean_score_prediction": 4.908380738153937,
    "group_id": "102292696432481"
  },
  {
    "image_id": "1634",
    "mean_score_prediction": 5.551215432868048,
    "group_id": "102292696432481"
  },
  {
    "image_id": "1635",
    "mean_score_prediction": 4.594676813456317,
    "group_id": "102292696432481"
  },
  {
    "image_id": "1636",
    "mean_score_prediction": 5.287141878007333,
    "group_id": "102292696432481"
  },
  {
    "image_id": "1637",
    "mean_score_prediction": 4.757534959644545,
    "group_id": "102292696432481"
  },
  {
    "image_id": "1638",
    "mean_score_prediction": 6.02659667759864,
    "group_id": "102292696432481"
  },
  {
    "image_id": "1639",
    "mean_score_prediction": 6.052004906422219,
    

In [42]:
results_array = [{"date": dt.utcnow(), **obj} for obj in lista]
results_array

[{'date': datetime.datetime(2023, 1, 18, 22, 36, 35, 489285),
  'image_id': '1631',
  'mean_score_prediction': 5.60638951636065,
  'group_id': '102292696432481'},
 {'date': datetime.datetime(2023, 1, 18, 22, 36, 35, 489288),
  'image_id': '1632',
  'mean_score_prediction': 5.142864143301267,
  'group_id': '102292696432481'},
 {'date': datetime.datetime(2023, 1, 18, 22, 36, 35, 489289),
  'image_id': '1633',
  'mean_score_prediction': 4.908380738153937,
  'group_id': '102292696432481'},
 {'date': datetime.datetime(2023, 1, 18, 22, 36, 35, 489290),
  'image_id': '1634',
  'mean_score_prediction': 5.551215432868048,
  'group_id': '102292696432481'},
 {'date': datetime.datetime(2023, 1, 18, 22, 36, 35, 489291),
  'image_id': '1635',
  'mean_score_prediction': 4.594676813456317,
  'group_id': '102292696432481'},
 {'date': datetime.datetime(2023, 1, 18, 22, 36, 35, 489292),
  'image_id': '1636',
  'mean_score_prediction': 5.287141878007333,
  'group_id': '102292696432481'},
 {'date': datetim

In [43]:
db["quality_assessment_results"].insert_many(results_array)

In [44]:
db.list_collection_names()

['quality_assessment_results', 'duplicated_images_model_results']

In [47]:
quality = db['quality_assessment_results']
df_quality = pd.DataFrame(quality.find())

In [48]:
df_quality.head()

,_id,date,image_id,mean_score_prediction,group_id
0,63c874b0af843974e5bd1823,2023-01-18 22:34:14.682,1631,5.606390,102292696432481
1,63c874b0af843974e5bd1824,2023-01-18 22:34:14.682,1632,5.142864,102292696432481
2,63c874b0af843974e5bd1825,2023-01-18 22:34:14.682,1633,4.908381,102292696432481
3,63c874b0af843974e5bd1826,2023-01-18 22:34:14.682,1634,5.551215,102292696432481
4,63c874b0af843974e5bd1827,2023-01-18 22:34:14.682,1635,4.594677,102292696432481
